# Project2 

### 1- Yazeed Ahmed Kamal 
### 2- Joseph Ayman 
### 3- Omar mohamed khaled 

# Preprocessing

- done in multiple steps in order to cover most of the cases to keep the data as clean as possible 

In [40]:
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

### Cleaning column name by removing spaces and capital letters

In [2]:
def clean_columname(taxi_df):
    #convert from string to datetime
    taxi_df['lpep pickup datetime'] = pd.to_datetime(taxi_df['lpep pickup datetime'])
    taxi_df['lpep dropoff datetime'] = pd.to_datetime(taxi_df['lpep dropoff datetime'])
    
    clean_data = taxi_df.copy()
    clean_data.columns = taxi_df.columns.str.replace(' ', '_').str.lower()
    
    return clean_data

### Removing duplicates in the data 

In [3]:
def remove_all_duplicates(taxi_df):
    clean_data = taxi_df.copy()
    #removing normal duplicates 
    clean_data = clean_data.drop_duplicates()
    #removing duplicates that appears as negative
    taxi_copy = clean_data.copy()
    num = taxi_copy.select_dtypes(include=['int', 'float'])
    for col in num:
        taxi_copy[col] = taxi_copy[col].abs()    
    duplicates = taxi_copy.duplicated()
    # Use boolean indexing to get the indices of duplicated rows
    duplicated_indices = taxi_copy.index[duplicates]
    clean_data = clean_data.drop(duplicated_indices)
    
    return clean_data

### Here cases means cases where the data rows has no logic in its features and values

In [4]:
def remove_diff_cases(taxi_df):
    #case 1
    clean_data = taxi_df.copy()
    neg_df = clean_data[clean_data['fare_amount'] < 0]
    ind = neg_df[( ((neg_df['payment_type'] == "Cash") | (neg_df['payment_type'] == "Credit Card") ) & (neg_df['trip_distance'] == 0))].index
    clean_data = clean_data.drop(ind)
    
    #case 2
    
    idx2 = neg_df[( ((neg_df['payment_type'] == "Cash") | (neg_df['payment_type'] == "Credit Card") ) & (neg_df['trip_distance'] != 0))].index
    numeric = clean_data.select_dtypes(include=['int', 'float'])
    for column in numeric:
        for index in idx2:
            if index in clean_data.index:  # Check if the index exists in the DataFrame
                clean_data.at[index, column] = abs(clean_data.at[index, column])
                
    #case 3
    
    No_charge_cond_index = clean_data[((clean_data['payment_type'] == 'No charge') & (clean_data['trip_distance'] == 0) & (clean_data['fare_amount'] < 0)) ].index
    columns_to_zero = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']

    for col in columns_to_zero:
        for i in No_charge_cond_index:
            clean_data.loc[i,col] = 0
            
    #case 4 
    
    dispute_ind = clean_data[((clean_data['fare_amount'] < 0) & (clean_data['payment_type'] == 'Dispute') & (clean_data['trip_distance'] == 0 ))].index
    clean_data = clean_data.drop(dispute_ind)
    
    #case 5
    
    copy = clean_data.copy()
    
    columns_to_sum = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge']
    copy['total_payment'] = copy[columns_to_sum].sum(axis=1) # Sum all except total
    
    diff = copy['total_payment'] - copy['total_amount']
    con_diff = diff > 0
    
    copy.loc[con_diff, 'total_amount'] = copy.loc[con_diff,'total_payment']
    clean_data = copy.drop('total_payment', axis=1)
    
    #drop because complete missing
    clean_data = clean_data.drop(['ehail_fee', 'congestion_surcharge' ], axis=1)
    
    return clean_data

## Adding a column representing the trip duration 

In [41]:
def add_columns(taxi_df):
    clean_data = taxi_df.copy()
    clean_data['trip_duration'] = clean_data['lpep_dropoff_datetime'] - clean_data['lpep_pickup_datetime']
    clean_data['trip_duration_hours'] = clean_data['trip_duration'].dt.total_seconds() / 3600  # Duration in hours
    
    return clean_data

### Handling data that contains missing entries (Nulls or NaNs)

In [5]:
def handle_missing(taxi_df):
    clean_data = taxi_df.copy()
    
    #passenger_count column------------------------------------------
    clean_data['passenger_count'].fillna(clean_data['passenger_count'].median(), inplace=True)
    #print("passengercount : ",clean_data['passenger_count'].isnull().sum() )
    
    #extra column---------------------------------
    pu_hours = clean_data['lpep_pickup_datetime'].dt.hour
    
    condition_one = ((clean_data['extra'].isnull()) & ((pu_hours == 16) | (pu_hours == 17) | (pu_hours == 18) | (pu_hours == 19)))
    # Use .loc to assign the value to the 'extra' column for rows that meet the condition
    clean_data.loc[condition_one, 'extra'] = 1
    
    condition_half = ((clean_data['extra'].isnull()) & ((pu_hours == 20) | (pu_hours == 21) | (pu_hours == 22) | (pu_hours == 23)))
    clean_data.loc[condition_half, 'extra'] = 0.5
    
    condition_else = (clean_data['extra'].isnull())
    clean_data.loc[condition_else, 'extra'] = clean_data.loc[condition_else, 'total_amount'] - clean_data.loc[condition_else, 'improvement_surcharge'] - clean_data.loc[condition_else, 'tolls_amount'] - clean_data.loc[condition_else, 'tip_amount']
   # print("extra : ", clean_data['extra'].isnull().sum())
    
    # payment type --------------------------------------------------------
    
    clean_data['payment_type'].fillna(value=clean_data['payment_type'].mode()[0], inplace=True)
    #print("payment_type", clean_data['payment_type'].isnull().sum())
    
    
    return clean_data

### Outliers Handling for different features 

In [39]:
def get_cutoff(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    cut_off = IQR * 2.5 
    lower = Q1 - cut_off
    upper =  Q3 + cut_off
    return lower,upper


def floor_and_cap(column):
    floor=column.quantile(0.1)
    cap=column.quantile(0.9)
    return floor,cap


def handle_outliers(taxi_df):
    clean_data = taxi_df.copy()
    
    # passenger_count outliers--------------------------------------
    clean_data = clean_data.drop(clean_data[clean_data['passenger_count'] > 6].index)
     
    
    #Trip_distance outliers ----------------------------------------
    lower, upper=get_cutoff(clean_data.trip_distance)
    df1_trip = clean_data[clean_data.trip_distance < lower]
    df2_trip = clean_data[clean_data.trip_distance > upper]
    
    floor, cap=floor_and_cap(clean_data.trip_distance)
    for i in df1_trip.index:
        clean_data.at[i, 'trip_distance'] = floor

    for i in df2_trip.index:
        clean_data.at[i, 'trip_distance'] = cap
        
      
    #fare_amount outliers -----------------------------------------
    clean_data = clean_data.drop(clean_data[clean_data.fare_amount > 200].index)
    
    clean_data['trip_duration_minutes'] = (clean_data['trip_duration'].dt.total_seconds() % 3600) / 60
    c = clean_data[((clean_data.fare_amount>0) & (clean_data.trip_distance==0) & ((clean_data.trip_duration_hours == 0) & (clean_data.trip_duration_minutes == 0) ))].index
    clean_data = clean_data.drop(c) 
    
    lower, upper=get_cutoff(clean_data.fare_amount)
    df1_fare = clean_data[clean_data.fare_amount < lower]
    df2_fare= clean_data[clean_data.fare_amount > upper]
    
    floor, cap=floor_and_cap(clean_data.fare_amount)
    for i in df1_fare.index:
        clean_data.at[i, 'fare_amount'] = floor

    for i in df2_fare.index:
        clean_data.at[i, 'fare_amount'] = cap
        
    
        
    
    #Extra outliers ----------------------------------------------------
    
    lower, upper=get_cutoff(clean_data.extra)
    df1_extra = clean_data[clean_data.extra > upper]
    df2_extra = clean_data[clean_data.extra < lower]
    
    floor, cap = floor_and_cap(clean_data.extra)

    for i in df1_extra.index:
        clean_data.at[i, 'extra'] = floor

    for i in df2_extra.index:
        clean_data.at[i, 'extra'] = cap
    
    
        
        
    #Tip_amount outliers-----------------------------
    lower, upper=get_cutoff(clean_data.tip_amount)
    df1_tip = clean_data[clean_data.tip_amount < lower]
    df2_tip = clean_data[clean_data.tip_amount > upper]
    
    floor, cap=floor_and_cap(clean_data.tip_amount)

    for i in df1_tip.index:
        clean_data.at[i, 'tip_amount'] = floor

    for i in df2_tip.index:
        clean_data.at[i, 'tip_amount'] = cap
        
    
        
    #Tolls_amount outliers------------------------------
    lower, upper=get_cutoff(clean_data.tolls_amount)
    df1_tolls = clean_data[clean_data.tolls_amount < lower]
    df2_tolls = clean_data[clean_data.tolls_amount > upper]
    
    clean_data = clean_data.drop(clean_data[clean_data.tolls_amount > 50].index)
    
    clean_data.loc[clean_data['tolls_amount'] > 10, 'tolls_amount'] = clean_data['tolls_amount'].median()
    
    
    
    #improvement_surcharge -----------------------------------------------
    clean_data = clean_data.drop(clean_data[clean_data.improvement_surcharge == -0.3].index)
    
    
    #Total_amount -------------------------------------------------------------
    columns_to_sum = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge']
    clean_data['total_amount'] = clean_data[columns_to_sum].sum(axis=1)
   
    
    
    
    return clean_data

### Encoding the categorical features to be able to use it in the model, we have used to types of encoding 
- **Label Encoding for**: pickup_loc_encoded, dropoff_loc_encoded, store_and_fwd_flag
- **One Hot Encoding for**: vendor, trip_type, payment_type, rate_type

In [7]:
def encoding(taxi_df):
    clean_data = taxi_df.copy()
    
    clean_data = pd.get_dummies(clean_data, columns=['vendor', 'trip_type', 'payment_type', 'rate_type'])
    
    label_encoder = LabelEncoder()
    clean_data['pickup_loc_encoded'] = label_encoder.fit_transform(clean_data['pu_location'])
    clean_data['dropoff_loc_encoded'] = label_encoder.fit_transform(clean_data['do_location'])
    clean_data['store_and_fwd_flag'] = label_encoder.fit_transform(clean_data['store_and_fwd_flag'])
    
    return clean_data

### Here we fill the lookup table which maps each original categorical value to its encoded feature 

In [8]:
def filling_lookup(df, col, lookup_table):
    encoded_values=preprocessing.LabelEncoder().fit_transform(df[col])
    map_values=pd.DataFrame({
    'Column Name' : [col] * len(encoded_values),
    'Original Value': df[col],
    'Imputed Value' : encoded_values})

    return pd.concat([lookup_table, map_values.drop_duplicates()])


def look_up(df):
    lookup_table = pd.DataFrame()
    lookup_table=filling_lookup(df, 'pu_location', lookup_table)
    lookup_table=filling_lookup(df, 'do_location', lookup_table)
    lookup_table=filling_lookup(df, 'store_and_fwd_flag', lookup_table)

    return lookup_table

###  Clean_dataset function includes all the functions above in one function to be easily called one time


In [44]:
def clean_dataset(taxi_df):
    after_naming = clean_columname(taxi_df)
    after_duplicates = remove_all_duplicates(after_naming )
    after_cases = remove_diff_cases(after_duplicates)
    after_adding_col = add_columns(after_cases)
    after_missing = handle_missing(after_adding_col)
    after_outliers = handle_outliers(after_missing)
    after_encoding = encoding(after_outliers)
    clean_data = after_encoding
    return clean_data

### This cleaning part without encoding will be used to get the correlation between the variables just not to get confused to find how the features affect the total amount which is the column of interest that will help us to prevent the models to overfit 


In [45]:
def clean_dataset_without_encoding(taxi_df):
    after_naming = clean_columname(taxi_df)
    after_duplicates = remove_all_duplicates(after_naming )
    after_cases = remove_diff_cases(after_duplicates)
    after_adding_col = add_columns(after_cases)
    after_missing = handle_missing(after_adding_col)
    after_outliers = handle_outliers(after_missing)
    clean_data = after_outliers
    return clean_data

# Reading uncleaned and then cleaning the dataset

In [46]:
taxi_df = pd.read_csv(r'green_tripdata_2019-11.csv')

In [47]:
taxi_df.head()

,Vendor,lpep pickup datetime,lpep dropoff datetime,store and fwd flag,Rate type,PU Location,DO Location,passenger count,trip distance,fare amount,extra,mta tax,tip amount,tolls amount,ehail fee,improvement surcharge,total amount,payment type,trip type,congestion surcharge
0,VeriFone Inc.,2019-11-01 00:11:24,2019-11-01 00:23:12,N,Standard rate,"Brooklyn,DUMBO/Vinegar Hill","Manhattan,Lower East Side",1.0,2.80,11.5,0.5,0.5,3.11,0.00,NaN,0.3,18.66,Credit card,Street-hail,2.75
1,VeriFone Inc.,2019-11-01 00:49:25,2019-11-01 01:14:19,N,Standard rate,"Queens,Long Island City/Hunters Point","Manhattan,Greenwich Village South",1.0,5.59,20.0,0.5,0.5,6.03,6.12,NaN,0.3,36.20,Credit card,Street-hail,2.75
2,"Creative Mobile Technologies, LLC",2019-11-01 00:57:22,2019-11-01 01:09:23,N,Standard rate,"Brooklyn,Williamsburg (North Side)","Brooklyn,Bushwick South",1.0,2.10,10.0,0.5,0.5,2.25,0.00,NaN,0.3,13.55,NaN,Street-hail,0.00
3,VeriFone Inc.,2019-11-01 00:59:52,2019-11-01 01:08:19,N,Standard rate,"Queens,Astoria","Queens,Sunnyside",1.0,1.23,7.5,0.5,0.5,0.00,0.00,NaN,0.3,8.80,Cash,Street-hail,0.00
4,VeriFone Inc.,2019-11-01 00:40:13,2019-11-01 00:47:41,N,Standard rate,"Queens,Jackson Heights","Queens,Jackson Heights",1.0,1.18,7.0,0.5,0.5,0.00,0.00,NaN,0.3,8.30,Cash,Street-hail,0.00


### Cleaned Without encoding

In [48]:
taxi_copy = taxi_df.copy()

In [49]:
taxi_copy['payment type'] = taxi_copy['payment type'].replace('Uknown', 'Unknown')

In [50]:
df_not_encoded = clean_dataset_without_encoding(taxi_copy)

In [52]:
df_not_encoded.drop(columns=['trip_duration','trip_duration_hours'], inplace=True)

In [53]:
df_not_encoded.head()

,vendor,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,rate_type,pu_location,do_location,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,trip_duration_minutes
0,VeriFone Inc.,2019-11-01 00:11:24,2019-11-01 00:23:12,N,Standard rate,"Brooklyn,DUMBO/Vinegar Hill","Manhattan,Lower East Side",1.0,2.80,11.5,0.5,0.5,3.11,0.00,0.3,15.91,Credit card,Street-hail,11.800000
1,VeriFone Inc.,2019-11-01 00:49:25,2019-11-01 01:14:19,N,Standard rate,"Queens,Long Island City/Hunters Point","Manhattan,Greenwich Village South",1.0,5.59,20.0,0.5,0.5,3.06,6.12,0.3,30.48,Credit card,Street-hail,24.900000
2,"Creative Mobile Technologies, LLC",2019-11-01 00:57:22,2019-11-01 01:09:23,N,Standard rate,"Brooklyn,Williamsburg (North Side)","Brooklyn,Bushwick South",1.0,2.10,10.0,0.5,0.5,2.25,0.00,0.3,13.55,Credit card,Street-hail,12.016667
3,VeriFone Inc.,2019-11-01 00:59:52,2019-11-01 01:08:19,N,Standard rate,"Queens,Astoria","Queens,Sunnyside",1.0,1.23,7.5,0.5,0.5,0.00,0.00,0.3,8.80,Cash,Street-hail,8.450000
4,VeriFone Inc.,2019-11-01 00:40:13,2019-11-01 00:47:41,N,Standard rate,"Queens,Jackson Heights","Queens,Jackson Heights",1.0,1.18,7.0,0.5,0.5,0.00,0.00,0.3,8.30,Cash,Street-hail,7.466667


### Copying the dataframe as csv to examine correlation for using it in the model diagnostics

In [54]:
year = 2019 
month = 11  
# Export the DataFrame to a CSV file 
filename = f"green_trip_data_{year}-{month:02d}_Clean_Not_Encoded.csv"
df_not_encoded.to_csv(filename, index=False)

## Cleaning the datafrom to enter it in the Models

In [55]:
cleaned_df = clean_dataset(taxi_copy)

In [56]:
cleaned_df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,pu_location,do_location,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,trip_duration_hours,trip_duration_minutes,"vendor_Creative Mobile Technologies, LLC",vendor_VeriFone Inc.,trip_type_Dispatch,trip_type_Street-hail,trip_type_Unknown,payment_type_Cash,payment_type_Credit card,payment_type_Dispute,payment_type_No charge,payment_type_Unknown,rate_type_Group ride,rate_type_JFK,rate_type_Nassau or Westchester,rate_type_Negotiated fare,rate_type_Newark,rate_type_Standard rate,rate_type_Unknown,pickup_loc_encoded,dropoff_loc_encoded
0,2019-11-01 00:11:24,2019-11-01 00:23:12,0,"Brooklyn,DUMBO/Vinegar Hill","Manhattan,Lower East Side",1.0,2.80,11.5,0.5,0.5,3.11,0.00,0.3,15.91,0 days 00:11:48,0.196667,11.800000,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,64,136
1,2019-11-01 00:49:25,2019-11-01 01:14:19,0,"Queens,Long Island City/Hunters Point","Manhattan,Greenwich Village South",1.0,5.59,20.0,0.5,0.5,3.06,6.12,0.3,30.48,0 days 00:24:54,0.415000,24.900000,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,207,124
2,2019-11-01 00:57:22,2019-11-01 01:09:23,0,"Brooklyn,Williamsburg (North Side)","Brooklyn,Bushwick South",1.0,2.10,10.0,0.5,0.5,2.25,0.00,0.3,13.55,0 days 00:12:01,0.200278,12.016667,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,100,54
3,2019-11-01 00:59:52,2019-11-01 01:08:19,0,"Queens,Astoria","Queens,Sunnyside",1.0,1.23,7.5,0.5,0.5,0.00,0.00,0.3,8.80,0 days 00:08:27,0.140833,8.450000,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,169,233
4,2019-11-01 00:40:13,2019-11-01 00:47:41,0,"Queens,Jackson Heights","Queens,Jackson Heights",1.0,1.18,7.0,0.5,0.5,0.00,0.00,0.3,8.30,0 days 00:07:28,0.124444,7.466667,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,200,201


In [57]:
cleaned_df.drop(columns=['lpep_pickup_datetime', 'lpep_dropoff_datetime','pu_location', 'do_location','trip_duration','trip_duration_hours'], inplace=True)

In [58]:
cleaned_df[['vendor_Creative Mobile Technologies, LLC',
       'vendor_VeriFone Inc.', 'trip_type_Dispatch', 'trip_type_Street-hail',
       'trip_type_Unknown', 'payment_type_Cash', 'payment_type_Credit card',
       'payment_type_Dispute', 'payment_type_No charge',
       'payment_type_Unknown', 'rate_type_Group ride', 'rate_type_JFK',
       'rate_type_Nassau or Westchester', 'rate_type_Negotiated fare',
       'rate_type_Newark', 'rate_type_Standard rate', 'rate_type_Unknown',
       'pickup_loc_encoded', 'dropoff_loc_encoded']] = cleaned_df[['vendor_Creative Mobile Technologies, LLC',
       'vendor_VeriFone Inc.', 'trip_type_Dispatch', 'trip_type_Street-hail',
       'trip_type_Unknown', 'payment_type_Cash', 'payment_type_Credit card',
       'payment_type_Dispute', 'payment_type_No charge',
       'payment_type_Unknown', 'rate_type_Group ride', 'rate_type_JFK',
       'rate_type_Nassau or Westchester', 'rate_type_Negotiated fare',
       'rate_type_Newark', 'rate_type_Standard rate', 'rate_type_Unknown',
       'pickup_loc_encoded', 'dropoff_loc_encoded']].astype(int)

In [59]:
cleaned_df.head()

,store_and_fwd_flag,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration_minutes,"vendor_Creative Mobile Technologies, LLC",vendor_VeriFone Inc.,trip_type_Dispatch,trip_type_Street-hail,trip_type_Unknown,payment_type_Cash,payment_type_Credit card,payment_type_Dispute,payment_type_No charge,payment_type_Unknown,rate_type_Group ride,rate_type_JFK,rate_type_Nassau or Westchester,rate_type_Negotiated fare,rate_type_Newark,rate_type_Standard rate,rate_type_Unknown,pickup_loc_encoded,dropoff_loc_encoded
0,0,1.0,2.80,11.5,0.5,0.5,3.11,0.00,0.3,15.91,11.800000,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,64,136
1,0,1.0,5.59,20.0,0.5,0.5,3.06,6.12,0.3,30.48,24.900000,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,207,124
2,0,1.0,2.10,10.0,0.5,0.5,2.25,0.00,0.3,13.55,12.016667,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,100,54
3,0,1.0,1.23,7.5,0.5,0.5,0.00,0.00,0.3,8.80,8.450000,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,169,233
4,0,1.0,1.18,7.0,0.5,0.5,0.00,0.00,0.3,8.30,7.466667,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,200,201


In [60]:
year = 2019 
month = 11  

# Export the DataFrame to a CSV file 
filename = f"green_trip_data_{year}-{month:02d}_Cleaned.csv"
cleaned_df.to_csv(filename, index=False)